In [ ]:
from google.colab import userdata
import os


NEBIUS_KEY = userdata.get("NEBIUS_API_KEY")
if not NEBIUS_KEY:
    raise RuntimeError("Add NEBIUS_API_KEY in Colab: Runtime → Variables (Secrets)")

os.environ["OPENAI_API_KEY"] = NEBIUS_KEY
print("NEBIUS_API_KEY loaded securely")


NEBIUS_API_KEY loaded securely


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    base_url="https://api.studio.nebius.com/v1",
    api_key=NEBIUS_KEY,
    temperature=0.0,
)
print("Nebius Chat Model Ready")


Nebius Chat Model Ready


In [ ]:
!pip install langchain langchain-openai langchain-community langchain-text-splitters chromadb sentence-transformers -q
print("LangChain + Embeddings Installed")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.8 MB/s eta 0:0

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

docs_text = [
    "Students must maintain 75% attendance in each course, except CIE. attendance doesnt matter for the CIE course",
    "students must File an FIR at the nearest policestation on losing thier ID cards",
    "Jio works from 02 to 05",
    "Medical exemptions must be submitted within 7 days of return."
]

emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectordb = Chroma.from_texts(docs_text, embedding=emb, collection_name="college_kb")
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

print("✅ Vector DB Created with", len(docs_text), "documents")


✅ Vector DB Created with 4 documents


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

SYSTEM = "Use ONLY the provided context. If not found, say you don't know."

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

def to_context(docs):
    # docs is a List[Document]; pull out the text
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": retriever | RunnableLambda(to_context), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
q = "is it fine if i have 65 percent attendance in CIE?"
print(rag_chain.invoke(q))

Yes, it is fine if you have 65 percent attendance in CIE.


In [ ]:
q1 = "whats the chairman's name"
print(rag_chain.invoke(q1))

I don't know. The provided context doesn't mention the chairman's name.


In [ ]:
q2 = "ive lost my ID what to do?"
print(rag_chain.invoke(q2))

You need to file an FIR.
